In [53]:
# Import necessary libraries
from ultralytics import YOLO
from PIL import Image

In [54]:
model = YOLO("weights/bestV4.pt")

In [55]:
image_path = 'training images/2.jpg'

In [56]:
results = model.predict(image_path, conf=0.05)
results[0].show()


image 1/1 c:\Users\samad\Coding\Tuwaiq\capstone-project-team01\training images\2.jpg: 640x640 16 blacks, 32 emptys, 13 whites, 233.7ms
Speed: 14.4ms preprocess, 233.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


In [57]:
# Function to get the board coordinates
def get_board_coordinates(xyxy):
    min_x = xyxy[:, 0].min().item()
    max_x = xyxy[:, 2].max().item()
    min_y = xyxy[:, 1].min().item()
    max_y = xyxy[:, 3].max().item()
    return min_x - 5, min_y - 5, max_x + 5, max_y + 5

In [58]:
xyxy = results[0].boxes.xyxy
min_x ,min_y ,max_x ,max_y = get_board_coordinates(xyxy)
print(f"Crop coordinates: ({min_x}, {min_y}), ({max_x}, {max_y})")

Crop coordinates: (333.1723937988281, 241.5214080810547), (2659.246826171875, 2708.675048828125)


In [59]:
img = Image.open(image_path)
width, height = img.size
print(width, ', ', height)

3024 ,  3024


In [60]:
lm = min_x
rm = width - max_x
bm = min_y
tm = height - max_y
print(tm, ', ', rm, ', ', bm, ', ', lm)


315.324951171875 ,  364.753173828125 ,  241.5214080810547 ,  333.1723937988281


In [61]:
new_shape = ((width - (lm + rm)), (height - (tm + bm)))
new_shape

(2326.074432373047, 2467.1536407470703)

In [62]:
# # Open the image using Pillow
# img = Image.open(image_path)

# # Crop the image
# cropped_img = img.crop((min_x, min_y, max_x, max_y))

# # Save the cropped image
# cropped_img.save('cropped_image.JPEG')
# print("Cropped image saved as 'cropped_image.JPEG'.")

In [63]:
# predict based on cropped
# results = model.predict(cropped_img, conf=0.05)
# results[0].show()

In [64]:
grid_rows = grid_cols = 8

# Create a grid for spaces (reversed order)
def create_grid(grid_rows, grid_cols):
    spaces = []
    for i in range(grid_rows):
        for j in range(grid_cols):
            spaces.append(f"{chr(72 - i)}{j + 1}")  # Start from 'H' (72 in ASCII)
    return spaces
spaces = create_grid(grid_rows, grid_cols)

In [75]:
# Map detections to cells (reversed grid)
def map_detections_to_spaces(boxes, spaces, classes, frame_shape, grid_rows, grid_cols):
    # Initialize all spaces to "initial"
    occupancy = {space: "initial" for space in spaces}
    index = 0
    row_ratio = grid_rows / frame_shape[0]
    col_ratio = grid_cols / frame_shape[1]

    row_size = 1 / row_ratio
    cells = []
    for box in boxes:
        # Calculate the center of the box
        # x_center = (box[0] + box[2]) / 2
        x_center = ((box[0] + box[2]) / 2) + ( 0.25 * row_size) - lm
        y_center = ((box[1] + box[3]) / 2) - bm


        # Calculate row and column indices
        row = int(y_center * row_ratio)
        col = int(x_center * col_ratio)

        # Clamp row and column indices to valid grid bounds
        row = max(0, min(row, grid_rows - 1))  # Ensure 0 <= row < grid_rows
        col = max(0, min(col, grid_cols - 1))  # Ensure 0 <= col < grid_cols

        # Map to a space identifier, reversed for rows
        space = f"{chr(72 - row)}{col + 1}"  # Reverse row index to start from 'H' (72)
        occupancy[space] = classes[index]
        cells.append(f'box{index}: class: {classes[index]}, space: {space}')
        index += 1
        
    return occupancy, cells
    

In [66]:
boxes = results[0].boxes.xyxy.cpu().numpy() if results else []

predicted_classes = results[0].boxes.cls  # `cls` contains the class indices
class_names = model.names  # Assuming the model has a `names` attribute for class labels
predicted_class_names = [class_names[int(cls_idx)] for cls_idx in predicted_classes]

In [77]:
occupancy, cells = map_detections_to_spaces(boxes, spaces, predicted_class_names, new_shape, 8, 8)
occupancy['B8']
for c in cells:
    if 'B' in c:
        print(c)
len(cells)

box46: class: white, space: B2
box48: class: white, space: B3
box50: class: empty, space: B6
box51: class: white, space: B1
box53: class: white, space: B8
box54: class: white, space: B4
box60: class: white, space: B5


61

In [68]:
def map_occupancy_to_board_status(occupancy):
    # Mapping rows and columns to a 2D array
    rows = ['H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']
    columns = ['1', '2', '3', '4', '5', '6', '7', '8']

    new_board_status = []

    for row in rows:
        row_data = []
        for col in columns:
            key = f"{row}{col}"
            cell_value = occupancy[key]
            row_data.append(cell_value)
        new_board_status.append(row_data)
    
    return new_board_status

new_board_status = map_occupancy_to_board_status(occupancy)

# Print the resulting 2D array
for row in new_board_status:
    print(row)

['black', 'black', 'black', 'black', 'black', 'black', 'black', 'black']
['black', 'black', 'black', 'black', 'empty', 'black', 'black', 'black']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
['empty', 'empty', 'empty', 'empty', 'black', 'empty', 'empty', 'empty']
['empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
['empty', 'empty', 'empty', 'empty', 'empty', 'initial', 'empty', 'empty']
['white', 'white', 'white', 'white', 'white', 'empty', 'initial', 'white']
['white', 'white', 'white', 'white', 'white', 'initial', 'white', 'white']
